In [3]:
from gzip import READ
from flask import Flask, request, render_template, redirect, url_for, flash, jsonify, send_from_directory, current_app
import requests
from string import Template
from flask_sqlalchemy import SQLAlchemy
from numpy import isnan
import sqlalchemy
import pandas as pd
pd.options.mode.chained_assignment = None
from pandas import DataFrame
from pandas.util import hash_pandas_object
import re
from flask_gtts import gtts
from config import GMAIL_PASSWORD, GMAIL_USERNAME, Config, S3_KEY, S3_SECRET, S3_BUCKET, SES_REGION_NAME, SES_EMAIL_SOURCE, GMAIL_USERNAME, GMAIL_PASSWORD, SERVER_NAME, SECRET_KEY
from flask_login import LoginManager
from models import LoginForm, RegistrationForm, PasswordResetForm, PasswordChangeForm, upload_file_to_s3, time_ago
from wtforms import validators
from wtforms.fields.html5 import EmailField
import email_validator
from passlib.hash import sha256_crypt
from flask_login import login_user, logout_user, login_required, current_user
import bcrypt
from datetime import datetime, timedelta
from dateutil import tz
from colour import Color
import re
import boto3, botocore
from django.utils.crypto import get_random_string
from werkzeug.utils import secure_filename
import os
import base64
import six
import uuid
import imghdr
import io
import pytz


# Not the entire world, just your best friends. 
application = Flask(__name__)
application.secret_key = application.config['SECRET_KEY']

application.config.from_object(Config)


#This is for users table
db = SQLAlchemy(application, engine_options={"pool_recycle": 1800})
db.init_app(application)

##Create SQL Engine Look at this: https://docs.sqlalchemy.org/en/14/core/pooling.html#pool-disconnects
engine = sqlalchemy.create_engine(application.config['SQLALCHEMY_DATABASE_URI'], pool_recycle=3600,)

In [12]:

class User(db.Model):

    __tablename__ = 'users'
    email = db.Column(db.String, primary_key=True)
    id = db.Column(db.String)
    handle = db.Column(db.String)
    password = db.Column(db.String)
    authenticated = db.Column(db.Boolean, default=False)
    
    def is_active(self):
        """True, as all users are active."""
        return True
    
    def get_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.email

    def get_user_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.id   

    def get_user_handle(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.handle         
        
    def is_authenticated(self):
        """Return True if the user is authenticate#d."""
        return self.authenticated


C:\Users\Dave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask_sqlalchemy\model.py:121: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  super(BindMetaMixin, cls).__init__(name, bases, d)


InvalidRequestError: Table 'users' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
df = get_feed(8, None)
df = format_feed(df)

print(df)

In [61]:
def notify_mentionted_users(post_text, current_user):
    if '@' in post_text:
        #Get the list of users that are mentioned
        users = re.findall(r'@([a-zA-Z0-9_]+)', post_text)
        for user in users:
                #Get user_id of follow account
            with engine.connect() as connection:
                ResultProxy = connection.execute('''SELECT u.id 
                                                    FROM users u 
                                                    WHERE u.handle = %s; ''', (user))

            df = DataFrame(ResultProxy.fetchall())
            if len(df) > 0:
                df.columns = ResultProxy.keys()

                mentioned_user_id = df['id'][0]
                print(mentioned_user_id)


                #Get the post_id of the post
                with engine.connect() as connection:
                    ResultProxy = connection.execute('''SELECT p.post_id
                                                        FROM posts p 
                                                        WHERE p.user_id = %s 
                                                        ORDER BY p.post_id DESC
                                                        LIMIT 1; ''', (current_user))
                    df = DataFrame(ResultProxy.fetchall())
                    df.columns = ResultProxy.keys()

                post_id = df['post_id'][0]

                #Notify the user that someone mentioned them
                event_type_id = 3
                with engine.connect() as connection:
                    connection.execute('INSERT INTO notifications (user_id, triggered_by_user_id, event_type_id, reference_post_id) VALUES (%s, %s, %s, %s);',  (mentioned_user_id, current_user, event_type_id, post_id))
                    
                        
            

In [ ]:

post_text = "TEST - @Matthew have you ever seen this? https://www.youtube.com/watch?v=dQw4w9WgXcQ"

current_user = 8

notify_mentionted_users(post_text, current_user)
    

In [ ]:
#Get the post_id of the post
with engine.connect() as connection:
    ResultProxy = connection.execute('''SELECT *
                                        FROM notifications
                                        ORDER BY id DESC; ''')
    df = DataFrame(ResultProxy.fetchall())
    df.columns = ResultProxy.keys()